### Download dataset 

In [0]:
import requests
import gzip
import shutil
import os
from pathlib import Path

# access url and city name
dbutils.widgets.text("city", "amsterdam") 
city = dbutils.widgets.get("city")
dbutils.widgets.text("url", "https://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2024-09-05/data/listings.csv.gz") 
url = dbutils.widgets.get("url")

download_path = "/Volumes/workspace/airbnb/airbnb//listings.csv.gz"
workspace_path = f"/Volumes/workspace/airbnb/airbnb/{city}.csv"

# Download the zipped raw data csv
print("Downloading file...")
response = requests.get(url, stream=True)
with open(download_path, "wb") as file:
    for chunk in response.iter_content(chunk_size=1024):
        file.write(chunk)
print("Download completed.")

# Extract the GZ file
print("Extracting file...")
with gzip.open(download_path, 'rb') as f_in:
    with open(workspace_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
print("Extraction completed.")

### View raw data

In [0]:
%sql
SELECT * FROM default.listings LIMIT 10

### Load listings.csv dataset and select useful feature columns

The following columns could be of use in the price prediction: 

| Column Name              | Example Value | Description |
|--------------------------|--------------|-------------|
| name                     | "Private, quiet studio in the centre with terrace"            | title of the airbnb page, should be transformed to embedding           |
| description              | "All guests agree: the apartment  is perfect and the location even better. A real home away from home. Two bedrooms, a fully equipped kitchen, a living with a comfortable couch. Quiet area, next to the Museumplein with the 3 major Museums."            | description on the airbnb page, should be transformed to embedding       |
| neighborhood_cleansed    |    Centrum-West       | label for the neighborhood, needs to be one-hot-encoded           |
| property_type           | Private room in guest suite            | label for the property type, needs to be one-hot-encoded           |
| room_type                | Entire home/apt            | label for the room type, needs to be one-hot-encoded           |
| accommodates            | 4            | the number of guests           |
| bathrooms               | 1            | the number of bathrooms           |
| bedrooms                | 2            | the number of bedrooms           |
| beds                    | 1            | the number of beds          |
| amenities               | ["Central heating", "Shower gel", "Lake access"]            | array of categorical variables, needs to be multi-hot-encoded           |
| availability_365        |   247          | number of days the airbnb is available per year           |
| review_scores_value     | 4.75            | review score for the value of the airbnb           |



In [0]:
import pandas as pd
import numpy as np

# Load the data into dataframe
df = spark.read.format("csv") \
.option("header", "true") \
.option("inferSchema", "true") \
.option("sep", ",") \
.option("escape", '"') \
.option("encoding", "UTF-8") \
.option("quote", '"') \
.option("multiLine", "true") \
.load("/Volumes/workspace/airbnb/airbnb/listings.csv").toPandas()

# Select specific columns
selected_columns = [
    "name", "description", "neighbourhood_cleansed",
    "property_type", "room_type", "accommodates", "bathrooms", "bathrooms_text", "bedrooms",
    "beds", "amenities", "availability_365", "review_scores_value", "price"
]

# Selecting the specified columns
df = df[selected_columns]

# Filter out records without price
df = df[df['price'].notna()]
df['price'] = df['price'].str.extract(r'(\d+\.?\d*)')
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Cutoff for cost/debug reasons
df = df.head(50)

# Display the first few rows
display(df)


### Clean columns one by one 

In [0]:
df['name'] = df['name'].fillna("")

In [0]:
df['description'] = df['description'].fillna("")

In [0]:
from sklearn.preprocessing import MinMaxScaler

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply Min-Max Scaling
df['accommodates'].fillna(1, inplace=True)
df['accommodates'] = scaler.fit_transform(df[['accommodates']])

In [0]:
df['bathrooms_text_num'] = df['bathrooms_text'].str.extract(r'(\d+\.?\d*)')
df['bathrooms_text_num'] = pd.to_numeric(df['bathrooms_text_num'], errors='coerce')
df['bathrooms'].fillna(df['bathrooms_text_num'], inplace=True)
df['bathrooms'].fillna(1, inplace=True)
df['bathrooms'] = scaler.fit_transform(df[['bathrooms']])

In [0]:
df['bedrooms'].fillna(df['accommodates'], inplace=True)
df['bedrooms'] = scaler.fit_transform(df[['bedrooms']])

In [0]:
df['beds'].fillna(df['bedrooms'], inplace=True)
df['beds'] = scaler.fit_transform(df[['beds']])

Here we select some amenities that are associated with higher prices and that occur reasonalby frequent. The others are removed

In [0]:
from collections import Counter
import pandas as pd
import numpy as np
import ast
from collections import defaultdict

# Flatten the list column and count occurrences
df['amenities'].fillna("[]", inplace=True)
df['amenities'] = df['amenities'].apply(ast.literal_eval) # only need to do this once, so if df state is preserved don't repeat this step
category_counts = Counter([item for sublist in df['amenities'] for item in sublist]).most_common()

# Look for amenities that indicate a high price
total_records = len(df)
amenity_price_sums = defaultdict(lambda: [0, 0])  # Format: {amenity: [total_price, count]}
for amenities, price in zip(df['amenities'], df['price']):
    for amenity in amenities:
        amenity_price_sums[amenity][0] += price  # Sum prices
        amenity_price_sums[amenity][1] += 1      # Count occurrences

# Filter amenities that appear in more than 5% of total records
threshold = 0.05 * total_records
filtered_amenities = {amenity: values for amenity, values in amenity_price_sums.items() if values[1] > threshold}

# Calculate average price for each filtered amenity
amenity_avg_prices = {amenity: total / count for amenity, (total, count) in filtered_amenities.items()}

# Sort by average price (descending)
sorted_amenity_avg_prices = sorted(amenity_avg_prices.items(), key=lambda x: x[1], reverse=True)

# ftake the first n items
n = 20
interesting_amenities = [label for label, avg_price in sorted_amenity_avg_prices][:n]

# Remove amenities that are not interesting
df['amenities'] = df['amenities'].apply(lambda x: [a for a in x if a in interesting_amenities])


In [0]:
df['availability_365'].fillna(df['availability_365'].mean(), inplace=True)
df['availability_365'] = df['availability_365'] / 365

In [0]:
df['review_scores_value'].fillna(df['review_scores_value'].mean(), inplace=True)
df['review_scores_value'] = df['review_scores_value'] / 5

### Save cleaned dataset

In [0]:
# Convert Pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(df)

# Save as Delta Table in Databricks
table_name = f"{city}_airbnb_dataset"
spark_df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(table_name)